In [ ]:
pip install pingouin
pip install scikit_posthocs

ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import scikit_posthocs as sp
import pingouin as pg

from scipy.stats import norm
from scipy.stats import ttest_rel, wilcoxon
from scipy.stats import kruskal
from scipy.stats import pearsonr

In [ ]:
"""
Transform a Qualtrics survey export into a long-form DataFrame.

Reads a semicolon-delimited CSV, retains metadata columns, maps each
question code to a numeric ID and sensitivity label, renames response
suffixes to descriptive names, and concatenates all questions into one
tidy DataFrame with a defined column order.
"""

import pandas as pd

df = pd.read_csv("Survey Results.csv", delimiter=";", encoding="utf-8")

meta_cols = [
    "ResponseId", "Status", "IPAddress", "Progress", "Finished",
    "LocationLatitude", "LocationLongitude", "Duration (in seconds)"
]

question_map = {
    "Q10": (0, "Non-sensitive"),
    "Q8":  (1, "Non-sensitive"),
    "Q9":  (2, "Non-sensitive"),
    "Q1":  (3, "Sensitive"),
    "Q3":  (4, "Sensitive"),
    "Q5":  (5, "Sensitive"),
    "Q6":  (6, "Sensitive"),
    "Q2":  (7, "Sensitive"),
    "Q4":  (8, "Sensitive"),
    "Q7":  (9, "Sensitive"),
}

suffix_map = {
    "-1_1": "SentimentConsistency",
    "-1_2": "FactualConsistency",
    "-2_1": "EnglishNeutrality",
    "-2_2": "ArabicNeutrality",
    "-2_3": "QuestionNeutrality",
    "-3":   "Comments",
}

rows = []
for code, (qid, sensitivity) in question_map.items():
    q_cols = [c for c in df.columns if c.startswith(f"{code}-")]
    if not q_cols:
        continue

    temp = df[meta_cols + q_cols].copy()
    rename_dict = {c: suffix_map[c.replace(code, "")] for c in q_cols}
    temp.rename(columns=rename_dict, inplace=True)
    temp["QuestionID"] = qid
    temp["Sensitivity"] = sensitivity
    rows.append(temp)

long_df = pd.concat(rows, ignore_index=True)

final_cols = (
    meta_cols
    + ["QuestionID", "Sensitivity"]
    + [
        "FactualConsistency",
        "SentimentConsistency",
        "EnglishNeutrality",
        "ArabicNeutrality",
        "QuestionNeutrality",
    ]
)
long_df = long_df[final_cols]

# print(long_df.head())
# print(long_df.shape, long_df.info())


In [ ]:
"""
Compute the mean survey duration after outlier removal.

This script:
1. Converts the 'Duration (in seconds)' column to numeric, coercing invalid entries to NaN.
2. Identifies outliers using the IQR method and a minimum threshold of 300 seconds.
3. Filters out those outliers.
4. Calculates and prints the mean duration, counts before and after filtering,
   and the IQR bounds used for filtering.
"""

df['Duration (in seconds)'] = pd.to_numeric(df['Duration (in seconds)'], errors='coerce')

# Extract unique duration values
tmp_df = df[['Duration (in seconds)']].drop_duplicates()

# Determine IQR bounds
Q1 = tmp_df['Duration (in seconds)'].quantile(0.25)
Q3 = tmp_df['Duration (in seconds)'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Keep durations ≥ 300 seconds and within IQR bounds
tmp_df_no_outliers = tmp_df[
    (tmp_df['Duration (in seconds)'] >= 300) &
    (tmp_df['Duration (in seconds)'] <= upper_bound)
]

mean_duration = tmp_df_no_outliers['Duration (in seconds)'].mean()
print(f"Mean Duration (in seconds) after removing outliers = {mean_duration:.2f}")

print(f"Unique durations before removal: {len(tmp_df)}")
print(f"Unique durations after removal: {len(tmp_df_no_outliers)}")
print(f"IQR lower bound: {lower_bound:.2f}, upper bound: {upper_bound:.2f}")


In [ ]:
"""
Filter and prepare the long-form survey DataFrame for analysis.

Steps:
1. Convert 'Progress' and 'Duration (in seconds)' to numeric.
2. Keep only completed responses with sufficient progress and duration.
3. Remove rows missing any key rating metrics.
4. Cast metadata and rating columns to appropriate types.
5. Report before/after row counts and unique IP–duration combinations, then display the first rows.
"""

# Convert columns to numeric types, coercing invalid entries
long_df["Progress"] = pd.to_numeric(long_df["Progress"], errors="coerce")
long_df["Duration (in seconds)"] = pd.to_numeric(long_df["Duration (in seconds)"], errors="coerce")

# Filter for valid, completed responses
valid = long_df.loc[
    (long_df["Status"] == "IP Address") &
    (long_df["Progress"] > 60) &
    (long_df["Duration (in seconds)"] > 300)
]

# Drop any row missing one of the five rating metrics
metrics = [
    "SentimentConsistency", "FactualConsistency",
    "EnglishNeutrality", "ArabicNeutrality", "QuestionNeutrality"
]
filtered_df = valid.dropna(subset=metrics).copy()

# Cast metadata columns to appropriate types
filtered_df.loc[:, "Status"] = filtered_df["Status"].astype(str)
filtered_df.loc[:, "IPAddress"] = filtered_df["IPAddress"].astype(str)
filtered_df.loc[:, "Progress"] = filtered_df["Progress"].astype(int)
filtered_df.loc[:, "Finished"] = filtered_df["Finished"].astype(bool)
filtered_df.loc[:, "Duration (in seconds)"] = filtered_df["Duration (in seconds)"].astype(int)
filtered_df.loc[:, "LocationLatitude"] = filtered_df["LocationLatitude"].astype(str)
filtered_df.loc[:, "LocationLongitude"] = filtered_df["LocationLongitude"].astype(str)

# Convert rating columns to integers
for col in metrics:
    filtered_df.loc[:, col] = filtered_df[col].astype(int)

# Print summary of filtering results
print("Rows before filtering:", long_df.shape[0])
print("Rows after filtering:", filtered_df.shape[0])
unique_pairs = filtered_df[["IPAddress", "Duration (in seconds)"]].drop_duplicates().shape[0]
print(f"Unique (IP Address, Duration) combinations: {unique_pairs}")
print(filtered_df.groupby("QuestionID").size())

filtered_df.head()


In [ ]:
"""
Generate overlapping violin plots comparing metric distributions for sensitive
and non-sensitive survey responses.

Defines a function to plot side-by-side violins for each metric and optionally
save the figure. Then demonstrates its use on the filtered DataFrame.
"""

def plot_group_violin_overlap(df, metrics, filename=None):
    """
    Create overlapping violin plots for two groups.

    Parameters
    ----------
    df : pandas.DataFrame
        Must contain a 'Sensitivity' column with values "Non-sensitive" and "Sensitive",
        and one column per metric in `metrics`.
    metrics : list of str
        Column names to plot on the x-axis.
    filename : str, optional
        If provided, path to save the figure (e.g. "overlap_violin.png").
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    positions = range(1, len(metrics) + 1)

    # Data for each group
    data_non = [df[df["Sensitivity"] == "Non-sensitive"][m].dropna().tolist() for m in metrics]
    data_sen = [df[df["Sensitivity"] == "Sensitive"][m].dropna().tolist() for m in metrics]

    vp_non = ax.violinplot(data_non, positions=positions, widths=0.6, showmedians=True)
    for part in vp_non['bodies']:
        part.set_alpha(0.5)

    vp_sen = ax.violinplot(data_sen, positions=positions, widths=0.6, showmedians=True)
    for part in vp_sen['bodies']:
        part.set_alpha(0.5)

    ax.set_xticks(positions)
    ax.set_xticklabels(metrics, rotation=45, fontsize=12)
    ax.set_ylabel("Score (5-point)", fontsize=12)
    ax.set_title(
        "Survey Metric Distributions — Sensitive vs Non-sensitive",
        fontsize=14, fontweight="bold"
    )
    ax.legend(
        [vp_non['bodies'][0], vp_sen['bodies'][0]],
        ["Non-sensitive", "Sensitive"],
        loc="upper left"
    )
    ax.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()

    if filename:
        fig.savefig(filename, dpi=300)

# Example usage:
plot_group_violin_overlap(filtered_df, metrics, filename="overlap_violin_final.png")


In [ ]:
"""
Plot histograms with normal density overlays for each survey metric.

For each metric in `metrics`, this script generates two side-by-side plots:
- Overall distribution with a fitted normal curve.
- Separate distributions for non-sensitive and sensitive responses.
"""

for metric in metrics:
    fig, axes = plt.subplots(ncols=2, figsize=(10, 4))

    # Overall distribution
    data_all = filtered_df[metric].dropna().astype(float)
    mu, sigma = data_all.mean(), data_all.std()
    x = np.linspace(data_all.min(), data_all.max(), 100)
    axes[0].hist(data_all, bins=5, density=True, alpha=0.6, edgecolor='black')
    axes[0].plot(x, norm.pdf(x, mu, sigma), 'k--')
    axes[0].set_title(f"{metric} — Overall")
    axes[0].set_xlabel("Rating")
    axes[0].set_ylabel("Density")

    # By sensitivity group
    for sens, color in zip(["Non-sensitive", "Sensitive"], ['#1f77b4', '#ff7f0e']):
        group = filtered_df[filtered_df["Sensitivity"] == sens][metric].dropna().astype(float)
        mu_g, sigma_g = group.mean(), group.std()
        axes[1].hist(group, bins=5, density=True, alpha=0.5, color=color, edgecolor='black')
        axes[1].plot(x, norm.pdf(x, mu_g, sigma_g), color=color, linestyle='--')
    axes[1].set_title(f"{metric} — By Sensitivity")
    axes[1].set_xlabel("Rating")
    axes[1].legend(["Non-sensitive", "Sensitive"])

    plt.tight_layout()
    plt.show()


In [ ]:
"""
For each survey metric, plot a 1×3 grid of:
- Histogram with fitted normal curve and Q–Q inset for all responses.
- The same for non-sensitive responses.
- The same for sensitive responses.
"""
for metric in metrics:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    groups = [
        ("Overall", filtered_df),
        ("Non-sensitive", filtered_df[filtered_df["Sensitivity"] == "Non-sensitive"]),
        ("Sensitive", filtered_df[filtered_df["Sensitivity"] == "Sensitive"])
    ]

    for ax, (label, subset) in zip(axes, groups):
        data = subset[metric].dropna().astype(float)
        mu, sigma = data.mean(), data.std()
        x = np.linspace(data.min(), data.max(), 100)

        ax.hist(data, bins=5, density=True, alpha=0.6, edgecolor="black")
        ax.plot(x, st.norm.pdf(x, mu, sigma), 'k--')

        qq = ax.inset_axes([0.6, 0.6, 0.35, 0.35])
        st.probplot(data, dist="norm", plot=qq)
        qq.set_title("Q–Q", fontsize=8)

        ax.set_title(f"{metric} — {label}")
        ax.set_xlabel("Rating")
        ax.grid(axis="y", linestyle="--", alpha=0.4)

    plt.tight_layout()
    plt.show()


In [ ]:
"""
Compute paired statistical tests and effect sizes for survey metrics.

Aggregates each respondent’s average ratings by sensitivity, then for each
metric performs paired t-tests, calculates Cohen’s d, and runs Wilcoxon tests
where applicable. Results are collected into a summary DataFrame.
"""

# Aggregate average ratings per respondent and sensitivity
agg = (
    filtered_df
    .groupby(["ResponseId", "Sensitivity"])[metrics]
    .mean()
    .unstack("Sensitivity")
)

results = []
for metric in metrics:
    # Extract paired non-sensitive and sensitive values
    non_vals = agg[(metric, "Non-sensitive")].dropna().astype(float).values
    sen_vals = agg[(metric, "Sensitive")].dropna().astype(float).values
    paired = pd.DataFrame({"non": non_vals, "sen": sen_vals}).dropna()
    non_vals = paired["non"].values
    sen_vals = paired["sen"].values

    # Paired t-test and Cohen’s d
    t_stat, t_p = ttest_rel(sen_vals, non_vals)
    cohens_d = (sen_vals - non_vals).mean() / (sen_vals - non_vals).std(ddof=1)

    # Wilcoxon signed-rank test if sufficient variation and sample size
    if len(paired) >= 10 and not np.allclose(sen_vals, non_vals):
        w_stat, w_p = wilcoxon(sen_vals, non_vals)
    else:
        w_stat, w_p = float("nan"), float("nan")

    results.append({
        "Metric":      metric,
        "Paired t":    round(t_stat, 3),
        "p (t-test)":  round(t_p, 4),
        "Cohen’s d":   round(cohens_d, 3),
        "Wilcoxon W":  w_stat,
        "p (Wilcoxon)": round(w_p, 4)
    })

# Create and display the comparison table
comparison_df = pd.DataFrame(results).set_index("Metric")
print(comparison_df)


In [ ]:
"""
Compute Kruskal–Wallis tests across questions for each metric within each sensitivity group.

For each sensitivity level and metric, this script:
1. Collects ratings by QuestionID.
2. Runs a Kruskal–Wallis test if there are at least two non-empty groups.
3. Aggregates H statistics and p-values into a DataFrame indexed by (Sensitivity, Metric).
"""

results = []
for sens in filtered_df["Sensitivity"].unique():
    for metric in metrics:
        groups = [
            filtered_df[
                (filtered_df["Sensitivity"] == sens) &
                (filtered_df["QuestionID"] == qid)
            ][metric].dropna().astype(float)
            for qid in filtered_df["QuestionID"].unique()
        ]
        groups = [g for g in groups if len(g) > 0]
        H, p = kruskal(*groups) if len(groups) > 1 else (None, None)
        results.append({
            "Sensitivity": sens,
            "Metric": metric,
            "H": round(H, 3) if H is not None else None,
            "p-value": round(p, 4) if p is not None else None
        })

question_level_df = pd.DataFrame(results).set_index(["Sensitivity", "Metric"])
print(question_level_df)


In [ ]:
"""
Plot mean survey ratings by question for each metric using grouped bars.

This script:
1. Calculates the mean of each metric grouped by QuestionID.
2. Displays a grouped bar chart where each question’s mean ratings are side-by-side
   for easy comparison across metrics.
"""

import numpy as np
import matplotlib.pyplot as plt

mean_table = filtered_df.groupby("QuestionID")[metrics].mean().astype(float)
n_questions = mean_table.shape[0]
bar_width = 0.8 / n_questions
x = np.arange(len(metrics))

fig, ax = plt.subplots(figsize=(12, 5))

for i, qid in enumerate(mean_table.index):
    ax.bar(
        x + i * bar_width,
        mean_table.loc[qid],
        width=bar_width,
        alpha=0.6,
        label=f"Q{qid}"
    )

ax.set_xticks(x + bar_width * (n_questions - 1) / 2)
ax.set_xticklabels(metrics, rotation=45, fontsize=12)
ax.set_ylabel("Mean rating (5-point)", fontsize=12)
ax.set_title("Mean Survey Ratings by Question within Each Metric",
             fontsize=14, fontweight="bold")
ax.grid(axis="y", linestyle="--", alpha=0.4)
ax.legend(title="Question ID", bbox_to_anchor=(1.02, 1), loc="upper left", borderaxespad=0)
plt.tight_layout()
plt.show()


In [ ]:
"""
Perform post-hoc Dunn’s tests for pairwise comparisons of question ratings.

For each sensitivity group ("Non-sensitive" and "Sensitive") and each metric,
runs Dunn’s test with Bonferroni-adjusted p-values across QuestionID groups,
and prints the resulting significance matrices.
"""

for sens in filtered_df["Sensitivity"].unique():
    print(f"\nPost-hoc Dunn’s test — {sens}")
    for metric in metrics:
        df_sub = (
            filtered_df[filtered_df["Sensitivity"] == sens]
            [["QuestionID", metric]]
            .dropna()
        )
        if df_sub["QuestionID"].nunique() > 1:
            dunn = sp.posthoc_dunn(
                df_sub,
                val_col=metric,
                group_col="QuestionID",
                p_adjust="bonferroni"
            )
            print(f"\n{metric}:\n{dunn.round(4)}")


In [ ]:
"""
Compute Pearson correlation matrices and plot heatmaps for survey metrics.

Defines:
- compute_corr_and_pvals(df, metrics): returns corr coefficients and p-values for given metrics.
- plot_corr_heatmap(corr, pvals, title, metrics): visualizes a correlation matrix with significance stars.

Then runs both overall and per-sensitivity analyses on `filtered_df` for the specified `metrics`.
"""

def compute_corr_and_pvals(df, metrics):
    corr = pd.DataFrame(index=metrics, columns=metrics, dtype=float)
    pvals = pd.DataFrame(index=metrics, columns=metrics, dtype=float)
    for i in metrics:
        for j in metrics:
            x = df[i].astype(float)
            y = df[j].astype(float)
            r, p = pearsonr(x, y)
            corr.loc[i, j] = round(r, 3)
            pvals.loc[i, j] = round(p, 4)
    return corr, pvals

def plot_corr_heatmap(corr, pvals, title, metrics):
    mask = np.eye(len(metrics), dtype=bool)
    fig, ax = plt.subplots(figsize=(7, 6))
    cmap = plt.cm.coolwarm
    im = ax.imshow(np.ma.masked_where(mask, corr), cmap=cmap, vmin=-1, vmax=1)

    # Annotate cells and add a star (★) if p-value < 0.05
    for i in range(len(metrics)):
        for j in range(len(metrics)):
            if not mask[i, j]:
                val = corr.iloc[i, j]
                text = f"{val:.2f}"
                if pvals.iloc[i, j] < 0.05:
                    text += "★"
                color = "white" if abs(val) > 0.3 else "black"
                ax.text(j, i, text, ha="center", va="center", color=color, fontsize=11)

    ax.set_xticks(np.arange(len(metrics)))
    ax.set_yticks(np.arange(len(metrics)))
    ax.set_xticklabels(metrics, rotation=45, ha="right", fontsize=12)
    ax.set_yticklabels(metrics, fontsize=12)
    ax.set_title(title, fontsize=16, pad=20)
    cbar = fig.colorbar(im, ax=ax, shrink=0.8, pad=0.02)
    cbar.set_label("Pearson r", fontsize=12)
    ax.grid(False)
    plt.tight_layout()
    plt.show()

corr_overall, pvals_overall = compute_corr_and_pvals(filtered_df, metrics)
print("Overall Correlation Matrix (r):\n", corr_overall)
print("\nOverall P-values:\n", pvals_overall)
plot_corr_heatmap(
    corr_overall,
    pvals_overall,
    "Overall Correlation Matrix of Survey Metrics ★ = p<0.05",
    metrics
)

for sens in filtered_df["Sensitivity"].unique():
    sens_df = filtered_df[filtered_df["Sensitivity"] == sens]
    corr_sens, pvals_sens = compute_corr_and_pvals(sens_df, metrics)
    print(f"\nCorrelation Matrix for: {sens} Questions (r):\n", corr_sens)
    print(f"\nP-values for Sensitivity = {sens}:\n", pvals_sens)
    plot_corr_heatmap(
        corr_sens,
        pvals_sens,
        f"Correlation Matrix for: {sens} Questions ★ = p<0.05",
        metrics
    )


In [ ]:
"""
Run paired Wilcoxon tests for neutrality and consistency metrics.

Compares ArabicNeutrality vs. EnglishNeutrality and SentimentConsistency vs. FactualConsistency
across three groups: overall, sensitive, and non-sensitive. Records test statistic W, p-value,
and rank-biserial correlation r for each comparison.
"""

comparisons = [
    ("ArabicNeutrality", "EnglishNeutrality", "Neutrality"),
    ("SentimentConsistency", "FactualConsistency", "Consistency")
]
groups = [
    ("Overall", filtered_df),
    ("Sensitive", filtered_df[filtered_df["Sensitivity"] == "Sensitive"]),
    ("Non-sensitive", filtered_df[filtered_df["Sensitivity"] == "Non-sensitive"])
]

results = []
for col1, col2, label in comparisons:
    for group_name, df_sub in groups:
        df_pair = df_sub[[col1, col2]].apply(pd.to_numeric, errors="coerce").dropna()
        if len(df_pair) >= 10:
            res = pg.wilcoxon(df_pair[col1], df_pair[col2])
            results.append({
                "Comparison": label,
                "Group": group_name,
                "W": round(res["W-val"].iloc[0], 3),
                "p-value": round(res["p-val"].iloc[0], 4),
                "r": round(res["RBC"].iloc[0], 3)
            })
        else:
            results.append({
                "Comparison": label,
                "Group": group_name,
                "W": None,
                "p-value": None,
                "r": None
            })

comparison_df = pd.DataFrame(results).set_index(["Comparison", "Group"])
print(comparison_df)

In [ ]:
"""
Perform paired t-tests for neutrality and consistency metrics across groups.

Compares:
- ArabicNeutrality vs. EnglishNeutrality
- SentimentConsistency vs. FactualConsistency

for each of:
- Overall responses
- Sensitive responses
- Non-sensitive responses

Outputs a table with T-statistics, p-values, and Cohen’s d, indexed by Comparison and Group.
"""

comparisons = [
    ("ArabicNeutrality", "EnglishNeutrality", "Neutrality"),
    ("SentimentConsistency", "FactualConsistency", "Consistency")
]
groups = [
    ("Overall", filtered_df),
    ("Sensitive", filtered_df[filtered_df["Sensitivity"] == "Sensitive"]),
    ("Non-sensitive", filtered_df[filtered_df["Sensitivity"] == "Non-sensitive"])
]

results = []
for col1, col2, label in comparisons:
    for group_name, df_sub in groups:
        df_pair = df_sub[[col1, col2]].apply(pd.to_numeric, errors="coerce").dropna()
        if len(df_pair) >= 10:
            res = pg.ttest(df_pair[col1], df_pair[col2], paired=True)
            results.append({
                "Comparison": label,
                "Group": group_name,
                "T": round(res["T"].iloc[0], 3),
                "p-value": round(res["p-val"].iloc[0], 4),
                "Cohen's d": round(res["cohen-d"].iloc[0], 3)
            })
        else:
            results.append({
                "Comparison": label,
                "Group": group_name,
                "T": None,
                "p-value": None,
                "Cohen's d": None
            })

comparison_df = pd.DataFrame(results).set_index(["Comparison", "Group"])
print(comparison_df)


In [ ]:
"""
Compute and visualize mean differences between survey metrics.

This module provides:
- compute_diff_means_and_pvals: calculate mean differences and p-values (paired t-test)
  between each pair of metrics.
- plot_diff_heatmap: display a heatmap of these mean differences with significance stars.

It then runs an overall analysis and repeats per sensitivity group.
"""

def compute_diff_means_and_pvals(df, metrics):
    """
    Compute mean difference and p-value matrices between metrics.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the metric columns.
    metrics : list of str
        Metric column names to compare.

    Returns
    -------
    diff_matrix : pandas.DataFrame
        Mean differences for each metric pair.
    pvals : pandas.DataFrame
        Paired t-test p-values for each metric pair.
    """
    diff_matrix = pd.DataFrame(index=metrics, columns=metrics, dtype=float)
    pvals = pd.DataFrame(index=metrics, columns=metrics, dtype=float)
    for i in metrics:
        for j in metrics:
            if i == j:
                diff_matrix.loc[i, j] = np.nan
                pvals.loc[i, j] = np.nan
            else:
                x = df[i].astype(float)
                y = df[j].astype(float)
                diff = np.mean(x - y)
                t, p = ttest_rel(x, y)
                diff_matrix.loc[i, j] = round(diff, 3)
                pvals.loc[i, j] = round(p, 4)
    return diff_matrix, pvals

def plot_diff_heatmap(diff_matrix, pvals, title, metrics):
    """
    Plot a heatmap of mean differences with significance annotations.

    Parameters
    ----------
    diff_matrix : pandas.DataFrame
        Matrix of mean differences between metrics.
    pvals : pandas.DataFrame
        Corresponding matrix of p-values.
    title : str
        Title for the heatmap.
    metrics : list of str
        Order of metric names for axes.
    """
    mask = np.eye(len(metrics), dtype=bool)
    fig, ax = plt.subplots(figsize=(7, 6))
    max_abs = np.nanmax(np.abs(diff_matrix.values))
    im = ax.imshow(
        np.ma.masked_where(mask, diff_matrix),
        cmap=plt.cm.coolwarm,
        vmin=-max_abs,
        vmax=max_abs
    )

    for i in range(len(metrics)):
        for j in range(len(metrics)):
            if not mask[i, j]:
                val = diff_matrix.iloc[i, j]
                text = f"{val:.2f}"
                if pvals.iloc[i, j] < 0.05:
                    text += "★"
                color = "white" if abs(val) > (0.3 * max_abs) else "black"
                ax.text(j, i, text, ha="center", va="center", color=color, fontsize=11)

    ax.set_xticks(np.arange(len(metrics)))
    ax.set_yticks(np.arange(len(metrics)))
    ax.set_xticklabels(metrics, rotation=45, ha="right", fontsize=12)
    ax.set_yticklabels(metrics, fontsize=12)
    ax.set_title(title, fontsize=16, pad=20)
    cbar = fig.colorbar(im, ax=ax, shrink=0.8, pad=0.02)
    cbar.set_label("Mean Difference", fontsize=12)
    ax.grid(False)
    plt.tight_layout()
    plt.show()

# Overall mean difference analysis
diff_overall, pvals_overall = compute_diff_means_and_pvals(filtered_df, metrics)
print("Overall Mean Difference Matrix:\n", diff_overall)
print("\nOverall P-values:\n", pvals_overall)
plot_diff_heatmap(
    diff_overall,
    pvals_overall,
    "Overall Mean Difference between Metrics ★ = p<0.05",
    metrics
)

# Per-sensitivity analysis
for sens in filtered_df["Sensitivity"].unique():
    sens_df = filtered_df[filtered_df["Sensitivity"] == sens]
    diff_sens, pvals_sens = compute_diff_means_and_pvals(sens_df, metrics)
    print(f"\nMean Difference Matrix for Sensitivity = {sens}:\n", diff_sens)
    print(f"\nP-values for Sensitivity = {sens}:\n", pvals_sens)
    plot_diff_heatmap(
        diff_sens,
        pvals_sens,
        f"Mean Difference between Metrics for Sensitivity: {sens} ★ = p<0.05",
        metrics
    )
